# Exploratory Data Analysis (Assignment # 4) - Team Forever Loop

**Authors:** Santosh Ganesan, Haran Nallasivan

**Date:** 23 September 2024

## Running this notebook

TODO: add instructions for running this notebook

## Introduction

TODO: talk about the data and the purpose of this exercise

## Data sourcing

TODO: imports and script to pull in data into dataframes/SQLite table as appropriate


In [11]:
# Import statements
import pandas as pd
from pathlib import Path

Path('./data').mkdir(exist_ok=True)

# Helper function for extracting and persisting data
def extract_df(csv_name):
    '''
    Extracts dataframe from the data set given a CSV filename. Persists locally to avoid redudant network calls and unnecessary load on Github.

    Parameters:
        csv_name (str) - CSV filename, e.g. "blood_pressure"
    Returns:
        A pandas dataframe
    '''
    try:
        return pd.read_csv(f'./data/{csv_name}.csv')
    except:
        df = pd.read_csv(f'https://raw.githubusercontent.com/Welltory/hrv-covid19/refs/heads/master/data/{csv_name}.csv')
        df.to_csv(f'./data/{csv_name}.csv')
        return df

# Loaading the data frames
csv_names = ['blood_pressure', 'heart_rate', 'hrv_measurements', 'participants', 'scales_description', 'sleep', 'surveys', 'wearables', 'weather']
df = {}
for csv_name in csv_names:
    df[csv_name] = extract_df(csv_name)
df['blood_pressure'].head(2)



## Preliminary analysis

TODO: blurb on what we'll do in this section

### Data structure

TODO: depict the data structure(s) and provide a description




### Descriptive statistics

TODO: Output some descriptive statistics and plots




### Data quality

TODO: preliminary data quality analysis (missing values, duplicates, outliers, variance)




### Variable relationships

TODO: correlation analysis between features of the data 




## Data cleaning

TODO: intro blurb on what we'll do in this section




### Handling missing values

TODO: remove missing values identified in previous data quality section




### Handling duplicates

TODO: remove duplicates identified in previous data quality section




### Handling outliers

TODO: remove outliers identified in previous data quality section




## Post-cleaning analysis

TODO: intro blurb on what we'll cover in this section

### A new feature

TODO: devise a new feature and analyze the feature




### Dimensionality reduction

TODO: employ a dimensionality reduction technique




### Data quality assessment

TODO: intro blurb on what we'll cover in this section




#### Data profiling

TODO: data profiling analysis




#### Data completeness

TODO: data completeness analysis




#### Data accuracy

TODO: data accuracy anaalysis


#### Data consistency

TODO: data consistency analysis



#### Data integrity

TODO: data integrity analysis




#### Data lineage

TODO: data lineage analysis




#### Data provenance

TODO: data provenance analysis




## Reference

Pravdin, Pavel (2022). *Welltory COVID-19 and Wearables Open Data Research [dataset].* 22 July 2022. Welltory Inc. CC0-1.0 License. https://github.com/Welltory/hrv-covid19